In [3]:
import os
import requests

# Replace with your Mistral AI API URL and API Key
MISTRAL_API_URL = "https://api.mistral.ai/v1/upload"  # Example URL, replace with actual
MISTRAL_API_KEY = "***REMOVED***"

# Path to the directory containing video files
VIDEO_PATH = 'tesla-real-world-video-q-a/videos/videos/'

# Headers for API authentication
headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

# Iterate over each file in the directory and upload if it's a .mp4 video
for video in os.listdir(VIDEO_PATH):
    if video.endswith('.mp4'):
        video_path = os.path.join(VIDEO_PATH, video)
        with open(video_path, 'rb') as video_file:
            files = {"file": video_file}
            try:
                # Make a POST request to upload the file
                response = requests.post(MISTRAL_API_URL, headers=headers, files=files)
                
                # Check for successful response
                if response.status_code == 200:
                    print(f"'{video}' uploaded successfully! Response: {response.json()}")
                else:
                    print(f"Failed to upload '{video}'. Error: {response.status_code}, {response.text}")
            except Exception as e:
                print(f"An error occurred while uploading '{video}': {e}")


Failed to upload '00001.mp4'. Error: 404, {
  "message":"no Route matched with those values",
  "request_id":"13c22f1d74a81132a1ba7d31c3920a23"
}
Failed to upload '00002.mp4'. Error: 404, {
  "message":"no Route matched with those values",
  "request_id":"797b1f585d7bda060aa02f54485ea93d"
}


KeyboardInterrupt: 

In [41]:
# CACHE THIS SO DONT HAVE TO UPLOAD EVERY TIME
video_names = ['/files/tw665rf6u2em', '/files/adot8mwqx5x8', '/files/9po1avlsm7f9', '/files/yt9a9psrka9r', '/files/p4qf8z7u09gi', '/files/tpen623uy5qr', '/files/cgwmuqbl16zr', '/files/e7hmaqm5i5ha', '/files/hklyvji2hmp7', '/files/iu1dm44rzjxb', '/files/zge9lpjs7h7q', '/files/zb88tzyjuznz', '/files/kwcwvib4tjpv', '/files/o8yib7ohyfxg', '/files/eopu01d6wp0j', '/files/ccske1io7v24', '/files/uxlhl1t9mpxc', '/files/kt7nxnixyfr9', '/files/7255bog4xq89', '/files/2nds4uxi92kd', '/files/esqldu9alume', '/files/ql30433b1vc6', '/files/9ez43wx70qdv', '/files/af34tanig9gc', '/files/pjd5wi63eb1u', '/files/bfwe4wfkjphm', '/files/ve24fszp6dgz', '/files/div8okijpkbh', '/files/splyrhu1g9xs', '/files/jztnog1jagxr', '/files/wqbstailjg60', '/files/dx7pwgwd110u', '/files/sp14h3tvduzy', '/files/n91m9yu0at27', '/files/wz80be455d0s', '/files/302jdw1o6y92', '/files/gknr0u24x64q', '/files/6qtaafyokqd0', '/files/h42kez3dh6lu', '/files/bo774opih334', '/files/f1gugtoepnxh', '/files/4b3rw572v53r', '/files/eglguei4pszy', '/files/1hiy0v2eki7q', '/files/l64min9tuwem', '/files/3wmvt82rh2nw', '/files/scdj8bb5rzc7', '/files/5tcacvfjio6t', '/files/r7f6i7w0ao7g', '/files/z8xnuwwlj74h']


In [4]:
import pandas as pd

questions = pd.read_csv('tesla-real-world-video-q-a/questions.csv')
questions['answers'] = None
questions.head()

,id,question,answers
0,1,Was ego doing a legal maneuver if its goal is ...,None
1,2,Where can ego legally park on this street? A. ...,None
2,3,What is the best description of the maneuver e...,None
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,None
4,5,"What is the blinker state of the oncoming car,...",None


In [8]:
import requests
import time
import csv

# ✅ Set up Mistral API Key
MISTRAL_API_KEY = "***REMOVED***"
API_URL = "https://api.mistral.ai/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

# ✅ Output CSV file
OUTPUT_CSV = "outputfile.csv"

def query_model(video_path, prompt):
    """
    Queries Mistral AI with a specific prompt.

    Args:
    - video_path (str): Path to the video (metadata used).
    - prompt (str): The question to be answered.

    Returns:
    - str: The generated response from Mistral AI.
    """
    video_metadata = f"Video filename: {video_path}"
    mistral_prompt = f"""
    First, summarize the key events from the video file: {video_metadata}.
    Then, analyze the following question and reason through the correct answer.
    
    Question: {prompt}
    Answer:
    """
    for attempt in range(3):  # Retry up to 3 times
        try:
            payload = {
                "model": "mistral-tiny",
                "messages": [
                    {"role": "system", "content": "You are an AI expert analyzing driving videos."},
                    {"role": "user", "content": mistral_prompt}
                ]
            }

            response = requests.post(API_URL, json=payload, headers=HEADERS)
            response.raise_for_status()  # Raise error if request fails

            return response.json()["choices"][0]["message"]["content"]

        except requests.exceptions.RequestException as e:
            print(f"⚠️ API Error: {e} | Retrying ({attempt+1}/3)...")
            time.sleep(5)  # Wait before retrying

    return "⚠️ Error: Could not process request after 3 attempts."

def process_videos_and_write_to_csv(video_folder, additional_question):
    """
    Process videos, query Mistral AI, and write results to a CSV file.

    Args:
    - video_folder (str): Path to the folder containing video files.
    - additional_question (str): A question to ask for each video.
    """
    # Collect results in a list
    results = []

    # Process each video
    for video in os.listdir(video_folder):
        if video.endswith(".mp4"):
            video_path = os.path.join(video_folder, video)
            
            print(f"🚀 Processing video: {video_path}...")

            # Main question response
            main_response = query_model(video_path, "What are the key events in this video?")
            
            # Additional question response
            additional_response = query_model(video_path, additional_question)

            # Append results to the list
            results.append({
                "video": video,
                "main_response": main_response,
                "additional_response": additional_response
            })

    # Write results to the output CSV file
    with open(OUTPUT_CSV, mode="w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["video", "main_response", "additional_response"])
        writer.writeheader()
        writer.writerows(results)

    print(f"✅ All results written to {OUTPUT_CSV}.")

# Main execution
if __name__ == "__main__":
    VIDEO_PATH = "tesla-real-world-video-q-a/videos/videos/"
    ADDITIONAL_QUESTION = "Does a car exist in this video?"
    process_videos_and_write_to_csv(VIDEO_PATH, ADDITIONAL_QUESTION)


🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00001.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00002.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00003.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00004.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00005.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00006.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00007.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00008.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00009.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00010.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00011.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00012.mp4...
🚀 Processing video: tesla-real-world-video-q-a/videos/videos/00013.mp4...
🚀 Processing video: tesla-real-world-v

In [ ]:
6. The video ends with the Tesla continuing its drive on the highway, still under autopilot control.","Yes, a car does exist in the video. The video file ""tesla-real-world-video-q-a/videos/videos/00005.mp4"" contains footage of a Tesla vehicle driving on a road. The Tesla is clearly visible throughout the video, making it the car in question."
00006.mp4,"The key events in the video file tesla-real-world-video-q-a/videos/videos/00006.mp4 are as follows:

1. The video begins with a Tesla Model S driving on a highway. The car appears to be traveling at a high speed.
2. The driver of the Tesla changes lanes to the right, signaling their intention to do so.
3. A tractor-trailer is seen approaching from the left in the next lane. The driver of the Tesla maintains a safe distance from the truck.
4. The tractor-trailer changes lanes to the right, coming closer to the Tesla. The Tesla driver quickly moves back to their original lane to avoid a collision.
5. The Tesla driver continues driving on the highway without any further incidents.


,id,question,answers
0,1,Was ego doing a legal maneuver if its goal is ...,None
1,2,Where can ego legally park on this street? A. ...,None
2,3,What is the best description of the maneuver e...,None
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,None
4,5,"What is the blinker state of the oncoming car,...",None
5,6,What is the reason ego changed lanes to the le...,None
6,7,What is the correct behavior for ego and why? ...,None
7,8,How many flashing traffic light bulbs are ther...,None
8,9,Which one if more accurate in terms of the dis...,None
9,10,"Ignoring pedestrian lights, how many traffic l...",None
